# Understanding Memory in LLMs

In the previous Notebooks, we successfully explored how OpenAI models can enhance the results from Azure AI Search queries. 

However, we have yet to discover how to engage in a conversation with the LLM. With [Bing Chat](http://chat.bing.com/), for example, this is possible, as it can understand and reference the previous responses.

There is a common misconception that LLMs (Large Language Models) have memory. This is not true. While they possess knowledge, they do not retain information from previous questions asked to them.

In this Notebook, our goal is to illustrate how we can effectively "endow the LLM with memory" by employing prompts and context.

In [1]:
import os
import random
from langchain_community.chat_message_histories import ChatMessageHistory, CosmosDBChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import ConfigurableFieldSpec
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter
from typing import List

from IPython.display import Markdown, HTML, display  

def printmd(string):
    display(Markdown(string))

#custom libraries that we will use later in the app
from common.utils import CustomAzureSearchRetriever, get_answer
from common.prompts import DOCSEARCH_PROMPT

from dotenv import load_dotenv
load_dotenv("credentials.env")

import logging

# Get the root logger
logger = logging.getLogger()
# Set the logging level to a higher level to ignore INFO messages
logger.setLevel(logging.WARNING)

In [2]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]

### Let's start with the basics
Let's use a very simple example to see if the GPT model of Azure OpenAI have memory. We again will be using langchain to simplify our code 

In [3]:
QUESTION = "Tell me some use cases for reinforcement learning"
FOLLOW_UP_QUESTION = "What was my prior question?"

In [4]:
COMPLETION_TOKENS = 1000
# Create an OpenAI instance
llm = AzureChatOpenAI(deployment_name=os.environ["GPT35_DEPLOYMENT_NAME"], 
                      temperature=0.5, max_tokens=COMPLETION_TOKENS)

In [5]:
# We create a very simple prompt template, just the question as is:
output_parser = StrOutputParser()
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an assistant that give thorough responses to users."),
    ("user", "{input}")
])

In [6]:
# Let's see what the GPT model responds
chain = prompt | llm | output_parser
response_to_initial_question = chain.invoke({"input": QUESTION})
display(Markdown(response_to_initial_question))

Reinforcement learning is a branch of machine learning that focuses on training agents to make optimal decisions in dynamic environments. It has a wide range of use cases across various domains. Here are some examples:

1. Game playing: Reinforcement learning has been successfully applied to games like chess, Go, and Atari games, where agents learn to play against human-level or even superhuman-level performance.

2. Robotics: Reinforcement learning can be used to train robots to perform complex tasks, such as grasping objects, navigating through environments, or even playing sports.

3. Autonomous vehicles: Reinforcement learning techniques can be employed to train self-driving cars to make decisions in real-time, such as lane changing, traffic signal recognition, and collision avoidance.

4. Finance: Reinforcement learning can be used in algorithmic trading to make optimal decisions about buying, selling, and holding financial assets based on market conditions and historical data.

5. Healthcare: Reinforcement learning can assist in personalized treatment planning by optimizing drug dosages, scheduling appointments, or identifying optimal treatment strategies for individual patients.

6. Energy management: Reinforcement learning can help optimize energy consumption in buildings by learning and adapting to occupancy patterns, weather conditions, and energy prices.

7. Resource allocation: Reinforcement learning can be used to optimize resource allocation in various domains, such as logistics, supply chain management, or network traffic routing.

8. Recommender systems: Reinforcement learning can be applied to personalize recommendations by learning from user feedback and adapting to individual preferences.

These are just a few examples, and the potential applications of reinforcement learning are vast and continue to expand as the field advances.

In [7]:
#Now let's ask a follow up question
printmd(chain.invoke({"input": FOLLOW_UP_QUESTION}))

Your prior question was "What was my prior question?"

As you can see, it doesn't remember what it just responded, sometimes it responds based only on the system prompt, or just randomly. This proof that the LLM does NOT have memory and that we need to give the memory as a a conversation history as part of the prompt, like this:

In [8]:
hist_prompt = ChatPromptTemplate.from_template(
"""
    {history}
    Human: {question}
    AI:
"""
)
chain = hist_prompt | llm | output_parser

In [9]:
Conversation_history = """
Human: {question}
AI: {response}
""".format(question=QUESTION, response=response_to_initial_question)

In [10]:
printmd(chain.invoke({"history":Conversation_history, "question": FOLLOW_UP_QUESTION}))

Your prior question was: "Tell me some use cases for reinforcement learning."

**Bingo!**, so we now know how to create a chatbot using LLMs, we just need to keep the state/history of the conversation and pass it as context every time

## Now that we understand the concept of memory via adding history as a context, let's go back to our GPT Smart Search engine

From Langchain website:
    
A memory system needs to support two basic actions: reading and writing. Recall that every chain defines some core execution logic that expects certain inputs. Some of these inputs come directly from the user, but some of these inputs can come from memory. A chain will interact with its memory system twice in a given run.

    AFTER receiving the initial user inputs but BEFORE executing the core logic, a chain will READ from its memory system and augment the user inputs.
    AFTER executing the core logic but BEFORE returning the answer, a chain will WRITE the inputs and outputs of the current run to memory, so that they can be referred to in future runs.
    
So this process adds delays to the response, but it is a necessary delay :)

![image](https://python.langchain.com/assets/images/memory_diagram-0627c68230aa438f9b5419064d63cbbc.png)

In [11]:
index1_name = "cogsrch-index-files"
index2_name = "cogsrch-index-books"
indexes = [index1_name, index2_name]

In [12]:
# Initialize our custom retriever 
retriever = CustomAzureSearchRetriever(indexes=indexes, topK=10, reranker_threshold=1)

If you check closely in prompts.py, there is an optional variable in the `DOCSEARCH_PROMPT` called `history`. Now it is the time to use it. It is basically a place holder were we will inject the conversation in the prompt so the LLM is aware of it before it answers.

**Now let's add memory to it:**

In [13]:
store = {} # Our first memory will be a dictionary in memory

# We have to define a custom function that takes a session_id and looks somewhere
# (in this case in a dictionary in memory) for the conversation
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


In [14]:
# We use our original chain with the retriever but removing the StrOutputParser
chain = (
    {
        "context": itemgetter("question") | retriever, 
        "question": itemgetter("question"),
        "history": itemgetter("history")
    }
    | DOCSEARCH_PROMPT
    | llm
)

## Then we pass the above chain to another chain that adds memory to it

output_parser = StrOutputParser()

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
) | output_parser

In [15]:
# This is where we configure the session id
config={"configurable": {"session_id": "abc123"}}

In [16]:
config

{'configurable': {'session_id': 'abc123'}}

Notice below, that we are adding a `history` variable in the call. This variable will hold the chat historywithin the prompt.

In [17]:
printmd(chain_with_history.invoke({"question": QUESTION}, config=config))

Reinforcement learning is a type of machine learning where an agent learns to make decisions by interacting with an environment and receiving feedback in the form of rewards or penalties. Here are some use cases for reinforcement learning:

1. **Game Playing**: Reinforcement learning has been successfully applied to playing complex games such as chess, Go, and poker. AlphaGo, developed by DeepMind, is a famous example of reinforcement learning used to defeat world champions in the game of Go.

2. **Robotics**: Reinforcement learning can be used to train robots to perform complex tasks and manipulate objects in real-world environments. For example, robots can learn to navigate through obstacles, grasp objects, or perform assembly tasks using reinforcement learning algorithms.

3. **Autonomous Vehicles**: Reinforcement learning can be used to train self-driving cars to make decisions in real-time, such as lane changing, merging, and following traffic rules. It can also help optimize fuel efficiency and reduce emissions.

4. **Resource Management**: Reinforcement learning can be used to optimize the allocation of resources in various domains, such as energy management, traffic control, and supply chain management. For example, it can help determine the optimal scheduling of power generation units or the routing of vehicles in a transportation network.

5. **Recommendation Systems**: Reinforcement learning can be used to personalize recommendations in e-commerce, online advertising, and content streaming platforms. It can learn user preferences and make recommendations that maximize user engagement and satisfaction.

6. **Healthcare**: Reinforcement learning can be used to develop personalized treatment plans, optimize drug dosage, and assist in medical diagnosis. It can learn from patient data and medical records to make informed decisions and improve patient outcomes.

7. **Finance and Trading**: Reinforcement learning can be used to develop trading strategies and make investment decisions. It can learn from historical market data and optimize trading policies to maximize returns and minimize risks.

8. **Control Systems**: Reinforcement learning can be used to control complex systems such as power grids, manufacturing processes, and autonomous drones. It can learn optimal control policies to achieve desired system behavior and performance.

These are just a few examples of the many potential use cases for reinforcement learning. The versatility and adaptability of reinforcement learning algorithms make them applicable to a wide range of domains and problems.

In [18]:
# Remembers
printmd(chain_with_history.invoke({"question": FOLLOW_UP_QUESTION},config=config))

Your prior question was: "Tell me some use cases for reinforcement learning."

In [19]:
# Remembers
printmd(chain_with_history.invoke({"question": "Thank you! Good bye"},config=config))

You're welcome! If you have any more questions in the future, feel free to ask. Goodbye and take care!

## Using CosmosDB as persistent memory

In previous cell we have added local RAM memory to our chatbot. However, it is not persistent, it gets deleted once the app user's session is terminated. It is necessary then to use a Database for persistent storage of each of the bot user conversations, not only for Analytics and Auditing, but also if we wish to provide recommendations in the future. 

Here we will store the conversation history into CosmosDB for future auditing purpose.
We will use a class in LangChain use CosmosDBChatMessageHistory

In [20]:
# Create the function to retrieve the conversation

def get_session_history(session_id: str, user_id: str) -> CosmosDBChatMessageHistory:
    cosmos = CosmosDBChatMessageHistory(
        cosmos_endpoint=os.environ['AZURE_COSMOSDB_ENDPOINT'],
        cosmos_database=os.environ['AZURE_COSMOSDB_NAME'],
        cosmos_container=os.environ['AZURE_COSMOSDB_CONTAINER_NAME'],
        connection_string=os.environ['AZURE_COMOSDB_CONNECTION_STRING'],
        session_id=session_id,
        user_id=user_id
        )

    # prepare the cosmosdb instance
    cosmos.prepare_cosmos()
    return cosmos


In [21]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name="User ID",
            description="Unique identifier for the user.",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="Unique identifier for the conversation.",
            default="",
            is_shared=True,
        ),
    ],
) | output_parser

In [22]:
# This is where we configure the session id and user id
random_session_id = "session"+ str(random.randint(1, 1000))
ramdom_user_id = "user"+ str(random.randint(1, 1000))

config={"configurable": {"session_id": random_session_id, "user_id": ramdom_user_id}}

In [23]:
config

{'configurable': {'session_id': 'session539', 'user_id': 'user629'}}

In [24]:
printmd(chain_with_history.invoke({"question": QUESTION}, config=config))

Reinforcement learning (RL) is a type of machine learning that involves an agent learning to make sequential decisions in an environment to maximize a reward signal. RL has been successfully applied to various domains and has several use cases, including:

1. **Game Playing**: RL has been used to train agents to play complex games such as chess, Go, and poker. For example, DeepMind's AlphaGo used RL techniques to defeat world champion Go players<sup><a href="https://deepmind.com/research/case-studies/alphago-the-story-so-far" target="_blank">[1]</a></sup>.

2. **Robotics**: RL is used to train robots to perform complex tasks and manipulate objects. For instance, RL has been used to teach robotic arms to grasp objects or navigate through an environment<sup><a href="https://arxiv.org/abs/1803.10122" target="_blank">[2]</a></sup>.

3. **Autonomous Vehicles**: RL can be used to train self-driving cars to make decisions in real-time, such as lane changing, merging, and navigating complex traffic scenarios<sup><a href="https://arxiv.org/abs/1904.06987" target="_blank">[3]</a></sup>.

4. **Recommendation Systems**: RL can be used to personalize recommendations for users in various domains, such as e-commerce, streaming platforms, and online advertising. The agent learns to optimize the recommendations based on user feedback and preferences<sup><a href="https://arxiv.org/abs/1812.10613" target="_blank">[4]</a></sup>.

5. **Resource Management**: RL can be used to optimize the allocation of resources in various domains, such as energy management, traffic control, and supply chain management. The agent learns to make decisions that maximize efficiency and minimize costs<sup><a href="https://arxiv.org/abs/1705.08439" target="_blank">[5]</a></sup>.

6. **Finance and Trading**: RL can be used to develop trading strategies and optimize portfolio management. The agent learns to make decisions based on market data and historical trends to maximize returns<sup><a href="https://arxiv.org/abs/1804.00360" target="_blank">[6]</a></sup>.

7. **Healthcare**: RL can be used to optimize treatment plans and personalize healthcare interventions. For example, RL has been used to develop personalized dosing strategies for cancer treatments<sup><a href="https://arxiv.org/abs/1902.03288" target="_blank">[7]</a></sup>.

These are just a few examples of the diverse range of applications for reinforcement learning. RL offers a powerful framework for training agents to make intelligent decisions in complex and dynamic environments.

In [25]:
# Remembers
printmd(chain_with_history.invoke({"question": FOLLOW_UP_QUESTION},config=config))

Your prior question was: "Tell me some use cases for reinforcement learning."

In [26]:
# Remembers
printmd(chain_with_history.invoke(
    {"question": "Can you tell me a one line summary of our conversation?"},
    config=config))

In our conversation, we discussed various use cases for reinforcement learning and its applications in game playing, robotics, autonomous vehicles, recommendation systems, resource management, finance and trading, and healthcare.

In [27]:
try:
    printmd(chain_with_history.invoke(
    {"question": "Thank you very much!"},
    config=config))
except Exception as e:
    print(e)

You're welcome! If you have any more questions, feel free to ask. I'm here to help!

In [28]:
printmd(chain_with_history.invoke(
    {"question": "I do have one more question, why did you give me a one line summary?"},
    config=config))

In [ ]:
printmd(chain_with_history.invoke(
    {"question": "why not 2?"},
    config=config))

#### Let's check our Azure CosmosDB to see the whole conversation


![CosmosDB Memory](./images/cosmos-chathistory.png)

# Summary
##### Adding memory to our application allows the user to have a conversation, however this feature is not something that comes with the LLM, but instead, memory is something that we must provide to the LLM in form of context of the question.

We added persitent memory using CosmosDB.

We also can notice that the current chain that we are using is smart, but not that much. Although we have given memory to it, it searches for similar docs everytime, regardless of the input. This doesn't seem efficient, but regardless, we are very close to finish our first RAG-talk to your data Bot.


## <u>Important Note</u>:<br>
As we proceed, while all the code will remain compatible with GPT-3.5 models (1106 or newer), we highly recommend transitioning to GPT-4. Here's why:

**GPT-3.5-Turbo** can be likened to a 7-year-old child. You can provide it with concise instructions, but it struggles sometimes to follow them accurately (not too reliable). Additionally, its limited "memory" (token context) can make sustained conversations challenging. Its response are also simple not deep.

**GPT-4-Turbo** exhibits the capabilities of a 10-12-year-old child. It possesses enhanced reasoning skills, consistently adheres to instructions and its answers are beter. It has extended memory retention (larger context size) for instructions, and it excels at following them. Its responses are deep and thorough.


# NEXT
We know now how to do a Smart Search Engine that can power a chatbot!! great!

In the next notebook 6, we are going to build our first RAG bot. In order to do this we will introduce the concept of Agents.